# 00 - Preparación de Datos

Este notebook maneja la descarga y preparación de datos:

1. **Descarga de LUNA16** (imágenes CT por subset)
2. **Verificación de LIDC-IDRI** (anotaciones de radiólogos via pylidc)
3. **Validación del overlap** entre datasets

## Datasets utilizados

| Dataset | Contenido | Tamaño | Fuente |
|---------|-----------|--------|--------|
| LUNA16 | Imágenes CT (.mhd/.raw) | ~13GB/subset | Zenodo |
| LIDC-IDRI | Anotaciones de 4 radiólogos | Incluido en pylidc | TCIA |

**Nota**: pylidc incluye una base de datos SQLite pre-construida con las anotaciones de los 1018 pacientes de LIDC-IDRI. No se requiere descargar archivos adicionales para las anotaciones.

In [1]:
# Configuracion inicial
import sys
import os

# Detectar entorno
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("[INFO] Ejecutando en Google Colab")
    
    # Instalar dependencias
    import subprocess
    paquetes = ['SimpleITK', 'pylidc', 'requests', 'tqdm']
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + paquetes)
    
    # Clonar repositorio
    repo_url = "https://github.com/Daspony/Imagenes-Biomedicas.git"
    if not os.path.exists("/content/Imagenes-Biomedicas"):
        subprocess.run(["git", "clone", repo_url], cwd="/content", check=True)
    sys.path.insert(0, "/content/Imagenes-Biomedicas")
    
    PROJECT_ROOT = "/content/Imagenes-Biomedicas"
else:
    print("[INFO] Ejecutando localmente")
    sys.path.append('..')
    PROJECT_ROOT = os.path.abspath('..')

print(f"[INFO] Directorio del proyecto: {PROJECT_ROOT}")

[INFO] Ejecutando localmente
[INFO] Directorio del proyecto: c:\Users\Poney\Desktop\Imagenes Biomedicas


In [2]:
# Importar modulos
from pathlib import Path
import numpy as np

# Modulos del proyecto
from utils.download_luna16 import download_luna16
from utils.lidc_loader import LIDCAnnotationLoader, verify_luna16_lidc_overlap

print("[OK] Modulos importados correctamente")

[OK] Modulos importados correctamente


## 1. Configuración de rutas

In [3]:
# Rutas principales
LUNA16_PATH = Path(PROJECT_ROOT) / 'LUNA16'

# Crear directorio si no existe
LUNA16_PATH.mkdir(parents=True, exist_ok=True)

print(f"[INFO] Directorio LUNA16: {LUNA16_PATH}")
print(f"       Existe: {LUNA16_PATH.exists()}")

[INFO] Directorio LUNA16: c:\Users\Poney\Desktop\Imagenes Biomedicas\LUNA16
       Existe: True


## 2. Descarga de LUNA16

Selecciona qué subsets descargar:
- `subsets=0`: Solo subset0 (~13GB) - Recomendado para empezar
- `subsets=[0,1,2]`: Múltiples subsets
- `subsets='all'`: Todos los subsets (~130GB)

In [ ]:
# Verificar estado actual
def verificar_subsets_existentes(luna_path):
    """Verifica que subsets ya estan descargados."""
    existentes = []
    for i in range(10):
        subset_path = luna_path / f'subset{i}'
        if subset_path.exists():
            mhd_files = list(subset_path.glob('*.mhd'))
            if mhd_files:
                existentes.append((i, len(mhd_files)))
    return existentes

existentes = verificar_subsets_existentes(LUNA16_PATH)

print("=" * 50)
print("ESTADO ACTUAL DE LUNA16")
print("=" * 50)

if existentes:
    print("\n[OK] Subsets encontrados:")
    for subset_id, num_files in existentes:
        print(f"     subset{subset_id}: {num_files} archivos .mhd")
else:
    print("\n[WARNING] No se encontraron subsets descargados, utiliza la siguiente celda para descargar.")

# Verificar CSVs
annotations_exists = (LUNA16_PATH / 'annotations.csv').exists()
candidates_exists = (LUNA16_PATH / 'candidates.csv').exists()
print(f"\n[{'OK' if annotations_exists else 'MISSING'}] annotations.csv")
print(f"[{'OK' if candidates_exists else 'MISSING'}] candidates.csv")

ESTADO ACTUAL DE LUNA16

[OK] Subsets encontrados:
     subset0: 89 archivos .mhd
     subset1: 89 archivos .mhd

[OK] annotations.csv
[OK] candidates.csv


In [9]:
# Configuracion de descarga
SUBSETS_A_DESCARGAR = [0,1,2,3]  # Cambiar a [0,1,2] o 'all' segun necesidad

# Ejecutar descarga si es necesario
def necesita_descarga(subsets, luna_path):
    """Verifica si se necesita descargar algun subset."""
    if subsets == 'all':
        subset_ids = list(range(10))
    elif isinstance(subsets, int):
        subset_ids = [subsets]
    else:
        subset_ids = subsets
    
    for sid in subset_ids:
        subset_path = luna_path / f'subset{sid}'
        if not subset_path.exists() or not list(subset_path.glob('*.mhd')):
            return True
    return False

if necesita_descarga(SUBSETS_A_DESCARGAR, LUNA16_PATH):
    print("[INFO] Iniciando descarga de LUNA16...")
    print("       (Esto puede tomar 30-60 minutos por subset)\n")
    
    success = download_luna16(
        subsets=SUBSETS_A_DESCARGAR,
        include_csv=True,
        download_dir=str(LUNA16_PATH)
    )
    
    if success:
        print("\n[OK] Descarga completada exitosamente")
    else:
        print("\n[ERROR] Error en la descarga")
else:
    print("[OK] Los datos solicitados ya estan descargados")

[OK] Los datos solicitados ya estan descargados


## 3. Verificación de anotaciones LIDC-IDRI

pylidc incluye una base de datos SQLite con las anotaciones de los 1018 pacientes de LIDC-IDRI.

In [10]:
# Inicializar cargador LIDC
print("Conectando a base de datos LIDC-IDRI...\n")
lidc_loader = LIDCAnnotationLoader(verbose=True)

# Obtener seriesuids de los subsets descargados
luna16_seriesuids = []

for subset_id, _ in verificar_subsets_existentes(LUNA16_PATH):
    subset_path = LUNA16_PATH / f'subset{subset_id}'
    uids = [f.stem for f in subset_path.glob('*.mhd')]
    luna16_seriesuids.extend(uids)
    print(f"subset{subset_id}: {len(uids)} seriesuids")

print(f"\nTotal seriesuids LUNA16: {len(luna16_seriesuids)}")

Conectando a base de datos LIDC-IDRI...

LIDC-IDRI database conectada: 1018 scans disponibles
subset0: 89 seriesuids
subset1: 89 seriesuids
subset2: 89 seriesuids
subset3: 89 seriesuids

Total seriesuids LUNA16: 356


In [11]:
# Verificar overlap con LIDC-IDRI
if luna16_seriesuids:
    print("\nVerificando overlap LUNA16 <-> LIDC-IDRI...\n")
    overlap_result = verify_luna16_lidc_overlap(luna16_seriesuids)
    
    print(f"\n[INFO] Resultado:")
    print(f"       {overlap_result['overlap_percentage']:.0f}% de scans tienen anotaciones LIDC")
else:
    print("[WARNING] No hay seriesuids para verificar (descarga datos primero)")


Verificando overlap LUNA16 <-> LIDC-IDRI...

=== LUNA16 <-> LIDC-IDRI Overlap ===
LUNA16 seriesuids: 356
LIDC-IDRI seriesuids: 1018
Overlap: 356 (100.0%)
Solo en LUNA16: 0
Solo en LIDC-IDRI: 662

[INFO] Resultado:
       100% de scans tienen anotaciones LIDC


## 4. Explorar anotaciones disponibles

In [13]:
# Explorar un caso de ejemplo
if luna16_seriesuids:
    example_uid = luna16_seriesuids[-1]
    print(f"Ejemplo: {example_uid[:50]}...\n")
    
    # Metadata
    metadata = lidc_loader.get_scan_metadata(example_uid)
    if metadata:
        print("Metadata del scan:")
        for key, value in metadata.items():
            print(f"   {key}: {value}")
    
    # Malignidad por consenso
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        consensus = lidc_loader.get_consensus_malignancy(example_uid)
    
    print(f"\nNodulos encontrados: {len(consensus)}")
    for nodule in consensus:
        print(f"   Nodulo {nodule['nodule_idx']}: "
              f"malignidad={nodule['malignancy_mean']:.2f} "
              f"({nodule['num_radiologists']} radiologos) - "
              f"{nodule['consensus_label']}")

Ejemplo: 1.3.6.1.4.1.14519.5.2.1.6279.6001.9754266256181847...

Metadata del scan:
   patient_id: LIDC-IDRI-0131
   pixel_spacing: 0.703125
   slice_thickness: 1.25
   slice_spacing: 1.25
   num_annotations: 4
   num_nodules: 2

Nodulos encontrados: 2
   Nodulo 0: malignidad=3.00 (1 radiologos) - Indeterminado
   Nodulo 1: malignidad=3.33 (3 radiologos) - Indeterminado


## Resumen

Este notebook preparo:

1. Descarga de LUNA16 (imagenes CT por subset)
2. Verificacion de anotaciones LIDC-IDRI disponibles

### Siguiente paso

Continua con **01_preprocesamiento.ipynb** para:
- Extraer mascaras de pulmon
- Extraer mascaras de nodulos (alineadas con LIDC)
- Normalizar imagenes CT